In [3]:
import neurokit2 as nk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from scipy.signal import butter, filtfilt

In [4]:
# Define a new function
def my_processing(ecg_signal,fs):
    # Do processing
    ecg_cleaned = nk.ecg_clean(ecg_signal, sampling_rate=fs, method="neurokit")
    instant_peaks, rpeaks, = nk.ecg_peaks(ecg_cleaned, sampling_rate=fs)
    rate = nk.ecg_rate(rpeaks, sampling_rate=fs, desired_length=len(ecg_cleaned))
    quality = nk.ecg_quality(ecg_cleaned, sampling_rate=fs)
    edr = nk.ecg_rsp(rate, sampling_rate=fs)

    # Prepare output
    signals = pd.DataFrame({"ECG_Raw": ecg_signal,
                            "ECG_Clean": ecg_cleaned,
                            "ECG_Rate": rate,
                            "ECG_Quality": quality,
                            "EDR": edr})
    signals = pd.concat([signals, instant_peaks], axis=1)
    info = rpeaks

    return signals, info

In [11]:
def get_ecg_features(subject,load_dir, save_dir):

    test_data = pd.read_csv(load_dir+'subject_'+subject+'_ecg.csv')

    print(test_data.head())
    fs = 256
    test_ecg = test_data.ECG

    ecg_proc,info = my_processing(test_ecg,fs)
    
    #epoch data  
    window = 30 
    intervals = np.arange(0, len(ecg_proc)-1, fs*window)
    epochs = nk.epochs_create(ecg_proc, events=intervals, sampling_rate=fs)

    epochs_df = nk.epochs_to_df(epochs)
    epochs_df = epochs_df[0:len(ecg_proc)] #fix issue with output size

    feat = nk.ecg_intervalrelated(epochs)

    #fix issues with output being in list

    columns_to_unbracket = feat.columns[2:]

    for col in columns_to_unbracket:
        feat[col] = feat[col].apply(lambda x: x[0][0])

    #labels 
    labels = test_data.psg_status.to_frame()
    epochs_df['labels'] = labels['psg_status']

    # binary classification
    epochs_df['labels2'] = labels['psg_status'].replace([1, 2, 3, 4, 5], 1)

    # 4 classes
    epochs_df['labels4'] = labels['psg_status'].replace([1,2], 1)
    epochs_df['labels4'] = epochs_df['labels4'].replace([3,4], 2)
    epochs_df['labels4'] = epochs_df['labels4'].replace([5], 3)
    
    #same for features 
    labels_trunc = labels.iloc[::256*30, :].reset_index()


    feat['labels'] = labels_trunc['psg_status'].values

    feat['labels2'] = feat['labels'].replace([1, 2, 3, 4, 5], 1).values

    feat['labels4'] = feat['labels'].replace([1,2], 1).values
    feat['labels4'] = feat['labels4'].replace([3,4], 2).values
    feat['labels4'] = feat['labels4'].replace([5], 3).values

    print(feat.head())
    print(epochs_df.head())

    feat.to_csv(save_dir+'subject_'+subject+'_ecg_feat.csv')
    epochs_df.to_csv(save_dir+'subject_'+subject+'_ecg_proc.csv')



In [12]:
subject = '0002'
load_dir = 'mesa_preproc/final/'
save_dir = ''

get_ecg_features(subject,load_dir, save_dir)

        ECG  session_id  psg_status
0 -0.555657           0           0
1 -0.561913           0           0
2 -0.585718           0           0
3 -0.581140           0           0
4 -0.541009           0           0


/Users/bojing/anaconda3/lib/python3.11/site-packages/neurokit2/hrv/hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(
/Users/bojing/anaconda3/lib/python3.11/site-packages/neurokit2/hrv/hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(
/Users/bojing/anaconda3/lib/python3.11/site-packages/neurokit2/complexity/entropy_multiscale.py:349: RuntimeWarning: invalid value encountered in scalar divide
  mse = np.trapz(mse) / len(mse)
/Users/bojing/anaconda3/lib/python3.11/site-packages/neurokit2/complexity/entropy

  Label  ECG_Rate_Mean  HRV_MeanNN   HRV_SDNN  HRV_SDANN1  HRV_SDNNI1  \
1     1      80.436524  192.307692  32.986563         NaN         NaN   
2     2      75.931858  201.864865   4.441985         NaN         NaN   
3     3      77.015533  199.270270   2.305171         NaN         NaN   
4     4      78.317247  196.000000   6.223893         NaN         NaN   
5     5      77.648963  198.162162   6.153199         NaN         NaN   

   HRV_SDANN2  HRV_SDNNI2  HRV_SDANN5  HRV_SDNNI5  ...  HRV_MSEn  HRV_CMSEn  \
1         NaN         NaN         NaN         NaN  ...  0.106996   0.120079   
2         NaN         NaN         NaN         NaN  ...       NaN        NaN   
3         NaN         NaN         NaN         NaN  ...  0.000000   0.816916   
4         NaN         NaN         NaN         NaN  ...  0.000000   0.000000   
5         NaN         NaN         NaN         NaN  ...  0.529440   0.632775   

   HRV_RCMSEn    HRV_CD   HRV_HFD   HRV_KFD   HRV_LZC  labels  labels2  \
1    0.11981